In [155]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

In [156]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV
def read_csvs():
    #import os
    #print( os.getcwd())
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()
has_dropped_keys = False # Para que no se droppeen cada vez que se ejecuta la celda siguiente

C:\Users\aleja\AppData\Local\Temp\ipykernel_12460\759616273.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


In [157]:
# Dropear IDs de todas las tablas (primera columna)
def drop_keys(dfs):
    for df_key in dfs.keys():
        if df_key not in ('hospital', 'patient'): # No dropear hospitalId o patientUnitStayId
            df = dfs[df_key]
            dfs[df_key] = df.drop(columns=[df.columns.values[0]])

if not has_dropped_keys:
    drop_keys(dfs)
    has_dropped_keys = True

In [158]:
def clean_labmeasurenamesystem(row, df):
    labmeasurenamesystem = row['labmeasurenamesystem']
        
    if labmeasurenamesystem != labmeasurenamesystem:
        return  df['labmeasurenameinterface']
    else:
        return labmeasurenamesystem 

In [159]:
# TODO: Mirar relación entre labmeasurenamesystem y labmeasurenameinterface DONE

dfs['lab']['labmeasurenamesystem']     = dfs['lab'].apply(lambda row : clean_labmeasurenamesystem(row, dfs['lab']), axis=1)

dfs['lab'] = dfs['lab'].drop(columns=[
    'labtypeid', # Irrelevante
    'labresulttext', # Redundante con labResult
    'labresultrevisedoffset', # Reundante con labresultoffset
    'labmeasurenameinterface'
], axis=1)

dfs['lab'] = dfs['lab'].dropna()

In [160]:
dfs['lab'].info()
print()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 431939 entries, 0 to 434659
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   patientunitstayid     431939 non-null  int64  
 1   labresultoffset       431939 non-null  int64  
 2   labname               431939 non-null  object 
 3   labresult             431939 non-null  float64
 4   labmeasurenamesystem  431939 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 19.8+ MB



In [163]:
query = 'select * from lab'

sql_query (query)

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset
0,437880563,1754323,-647,3,Hct,38.3,38.3,%,%,-631
1,437880572,1754323,-647,3,platelets x 1000,181,181,K/mcL,k/mm cu,-631
2,437880560,1754323,-647,3,RBC,4.86,4.86,M/mcL,m/mm cu,-631
3,437880570,1754323,-647,3,-monos,8.7,8.7,%,%,-631
4,437880571,1754323,-647,3,MCHC,30.4,30.4,g/dL,%,-631
...,...,...,...,...,...,...,...,...,...,...
434655,672795149,2754778,2041,4,bedside glucose,91,91,mg/dL,mg/dL,2041
434656,641165125,2754778,37,1,troponin - I,5.92,5.920,ng/mL,ng/mL,37
434657,667153606,2754778,37,3,MCV,92,92,fL,fL,37
434658,673870691,2754778,236,4,bedside glucose,119,119,mg/dL,mg/dL,236


In [162]:
dfs['lab'] 

,patientunitstayid,labresultoffset,labname,labresult,labmeasurenamesystem
0,1754323,-647,Hct,38.30,%
1,1754323,-647,platelets x 1000,181.00,K/mcL
2,1754323,-647,RBC,4.86,M/mcL
3,1754323,-647,-monos,8.70,%
4,1754323,-647,MCHC,30.40,g/dL
...,...,...,...,...,...
434655,2754778,2041,bedside glucose,91.00,mg/dL
434656,2754778,37,troponin - I,5.92,ng/mL
434657,2754778,37,MCV,92.00,fL
434658,2754778,236,bedside glucose,119.00,mg/dL
